In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

data_list = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data_list.append(os.path.join(dirname, filename))

        
# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


> First, I will check the filenames that already listed

In [15]:
# Check filenames
data_list

['/kaggle/input/titanic/train.csv',
 '/kaggle/input/titanic/gender_submission.csv',
 '/kaggle/input/titanic/test.csv']

> Make each dataframe with the input from the above filenames

In [16]:
# Create dataframes from csv
titanic_train = pd.read_csv(data_list[0])
titanic_gendersub = pd.read_csv(data_list[1])
titanic_test = pd.read_csv(data_list[2])

> Get the knowledges from data. I must get any information by processing the data

In [17]:
# Inspect the data
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [66]:
# Detecting Null Values
# This will determine which column I will use
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [74]:
# Unique Values
# Same with above, by knowing the unique values. I can tell which feature is important
for name in titanic_train:
    print(f"Unique values of {name} is", 
          "Important v" if len(titanic_train[name].unique()) < 10 else "Not Important")

Unique values of PassengerId is Not Important
Unique values of Survived is Important v
Unique values of Pclass is Important v
Unique values of Name is Not Important
Unique values of Sex is Important v
Unique values of Age is Not Important
Unique values of SibSp is Important v
Unique values of Parch is Important v
Unique values of Ticket is Not Important
Unique values of Fare is Not Important
Unique values of Cabin is Not Important
Unique values of Embarked is Important v


> I should know the surviving rate of male and females

In [76]:
# Women rate of surviving
women = titanic_train.loc[titanic_train.Sex == 'female']["Survived"]
women_rate = (sum(women) / len(women)) * 100
print("{0:.2f}% women survived the shipwreck".format(women_rate))

74.20% women survived the shipwreck


In [77]:
# Men rate of surviving
men = titanic_train.loc[titanic_train.Sex == 'male']["Survived"]
men_rate = (sum(men) / len(men)) * 100
print("{0:.2f}% men survived the shipwreck".format(men_rate))

18.89% men survived the shipwreck


> As I know the summarization of the data. I can begin the prediction

In [ ]:
# Begin Machine Learning Modeling
# 1. Random Forest
from sklearn.ensemble import RandomForestClassifier

y = titanic_train["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]

X = pd.get_dummies(titanic_train[features])

X_test = pd.get_dummies(titanic_test[features])

In [82]:
forest = RandomForestClassifier(n_estimators=100, max_depth=None, max_features=2, random_state=0)
forest_prediction = forest.fit(X, y)
print( "Accuracy on training set {:.2f}%".format(forest_prediction.score(X, y)*100) )

Accuracy on training set 81.71%


> Allright, it seems my accuracy is ceil at 81.71%. I will try another method.

In [89]:
# 2. Gradient Boosting Regression Tree
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=0, learning_rate=1, max_depth=5)

gradient_prediction = gbrt.fit(X, y)
print( "Accuracy on training set {:.2f}%".format(gradient_prediction.score(X, y)*100) )



Accuracy on training set 81.71%


> Same as above, maybe I will end my experiment here and submit it

In [90]:
real_prediction = gbrt.predict(X_test)
output = pd.DataFrame({"PassengerId": titanic_test.PassengerId, 'Survived':real_prediction})
output.to_csv("my_submission.csv", index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
